This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [1]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn


import utils
from gsp_utils.baselines_archs import GCNN_2L, MLP, GAT
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows, Dual_GFGCN 

# SEED = 0
SEED = 15
PATH = 'results/try_datasets'
SAVE = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:0


In [2]:
def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

def summary_table(accs, datasets, exps, median=False):
    mean_accs = accs.mean(axis=2)
    cols_name = []
    for dataset_name in datasets:
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])
        cols_name.append(f'{dataset_name} ({edge_hom:.2f})')

    index_name = [exp['leg'] for exp in exps]

    return DataFrame(mean_accs, columns=cols_name, index=index_name)


In [3]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'ChameleonDataset', 'CoraGraphDataset', 'CiteseerGraphDataset']
# DATASETS = ['CiteseerGraphDataset', 'CoraGraphDataset'. 'ChameleonDataset', 'ActorDataset']
# DATASETS = ['ActorDataset']


## Best params

In [4]:
N_RUNS = 10
ACT = nn.ReLU()  # nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss() #nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-A'},

        # {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
        #   'hid_dim': 16, 'leg': 'MLP'},
        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'hid_dim': 32, 'leg': 'MLP'},

        {'model': 'GAT', 'heads': 2, 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': 0,
          'hid_dim': 16, 'leg': 'GAT'},
          

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50, 'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'A-GCN-normA'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'A-GCN'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'A-GCN-v2'},

        {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .005, 'wd': 5e-4, 'drop': .5,
         'hid_dim': 32,'L': 3, 'K': 2, 'norm': True, 'leg': 'H-GCN-normH'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 50, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'bias': True, 'alpha': None, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN-alpha'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'bias': True, 'alpha': None, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'Dual-GFCN-alpha-n'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
          'bias': False, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
          'bias': True, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': .1, 'norm': True, 'leg': 'Dual-GFCN-n'},

        # {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 32,'L': 2, 'K': 3, 'norm': False, 'leg': 'H-GCN'},
         

        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN'},
      ]


In [5]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, exp['hid_dim'], out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=exp['drop'], norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'MLP':
                arch = MLP(in_dim,  exp['hid_dim'], out_dim, dropout=exp['drop'])

            elif exp['model'] == 'GAT':
                gat_params = {'attn_drop': exp['drop']}
                arch = GAT(in_dim,  exp['hid_dim'], out_dim, exp['heads'], gat_params, act=ACT, last_act=LAST_ACT)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=exp['drop'], diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=exp['drop'], norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=exp['drop'], diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'Dual-GFGCN':
                arch = Dual_GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, alpha=exp['alpha'],
                                  last_act=LAST_ACT, dropout=exp['drop'], init_h0=exp['h0'], bias=exp['bias'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'MLP', 'GAT', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'],
                                        epochs_h=exp['e_h'], epochs_W=exp['e_W'])
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc1 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val1 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val1b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 

/home/srey/Investigacion/robust_minmax_gnn/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


0.622 (0.676) - 0.730 (0.730) - 0.838 (0.838) - 0.595 (0.676) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:44: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(np.where(np.isclose(deg_vec, 0), 0, 1/deg_vec))


0.784 (0.865) - 0.811 (0.892) - 0.838 (0.946) - 0.811 (0.865) - 0.865 (0.892) - 0.838 (0.865) - 0.865 (0.892) - 0.838 (0.865) - 0.784 (0.838) - 0.865 (0.865) - 
1: 0.568 (0.622) - 0.622 (0.703) - 0.757 (0.865) - 0.595 (0.649) - 0.919 (0.946) - 0.973 (0.973) - 0.919 (1.000) - 0.946 (0.973) - 0.973 (0.973) - 0.946 (0.946) - 0.973 (0.973) - 0.865 (0.946) - 0.811 (0.892) - 0.811 (0.838) - 
2: 0.486 (0.541) - 0.486 (0.595) - 0.757 (0.784) - 0.514 (0.541) - 0.865 (0.865) - 0.838 (0.919) - 0.757 (0.892) - 0.811 (0.919) - 0.811 (0.865) - 0.838 (0.892) - 0.838 (0.892) - 0.784 (0.865) - 0.757 (0.757) - 0.730 (0.784) - 
3: 0.622 (0.649) - 0.676 (0.703) - 0.838 (0.865) - 0.595 (0.649) - 0.811 (0.946) - 0.838 (0.919) - 0.892 (0.946) - 0.892 (0.919) - 0.838 (0.946) - 0.892 (0.946) - 0.919 (0.946) - 0.892 (0.946) - 0.838 (0.892) - 0.757 (0.838) - 
4: 0.486 (0.595) - 0.595 (0.676) - 0.730 (0.838) - 0.568 (0.622) - 0.811 (0.892) - 0.784 (0.892) - 0.811 (0.892) - 0.865 (0.919) - 0.919 (0.973) - 0.892 (0

In [6]:
if SAVE:
    timestr = time.strftime("%Y%m%d-%H%M")
    table_acc_val1.to_csv(PATH + 'best_params_val_test_' + timestr)
    table_acc_val1.to_csv(PATH + 'best_params_best_val_' + timestr)

In [7]:
table_acc_val1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.591892,0.580392,0.459459,0.610526,0.4848,0.5903
Kipf-A,0.572973,0.574510,0.437838,0.346491,0.7813,0.6765
MLP,0.781081,0.835294,0.735135,0.482018,0.5991,0.5955
GAT,0.608108,0.517647,0.410811,0.611184,0.5917,0.6477
A-GCN-normA,0.837838,0.860784,0.772973,0.470833,0.7994,0.6907
A-GCN,0.837838,0.856863,0.764865,0.439693,0.7541,0.6722
A-GCN-v2,0.816216,0.868627,0.794595,0.450658,0.7639,0.6767
H-GCN-normH,0.848649,0.854902,0.770270,0.435746,0.7932,0.6781
Dual-GFCN-alpha,0.827027,0.856863,0.751351,0.458991,0.7633,0.6777
Dual-GFCN-alpha-n,0.854054,0.860784,0.764865,0.472588,0.7999,0.6878


In [8]:
table_acc_val1b

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.591892,0.572549,0.451351,0.616228,0.4841,0.5904
Kipf-A,0.567568,0.574510,0.437838,0.344956,0.7841,0.6813
MLP,0.781081,0.819608,0.737838,0.500000,0.5886,0.5896
GAT,0.608108,0.531373,0.416216,0.609430,0.6111,0.6704
A-GCN-normA,0.827027,0.860784,0.754054,0.462939,0.7986,0.6855
A-GCN,0.827027,0.864706,0.772973,0.439912,0.7514,0.6632
A-GCN-v2,0.824324,0.852941,0.759459,0.457675,0.7646,0.6739
H-GCN-normH,0.835135,0.858824,0.748649,0.432237,0.7959,0.6839
Dual-GFCN-alpha,0.802703,0.847059,0.743243,0.462061,0.7637,0.6770
Dual-GFCN-alpha-n,0.859459,0.849020,0.762162,0.473684,0.7988,0.6851


## Initial Params

In [9]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 10
N_EPOCHS = 200  # 500
LR = .01
WD = 5e-4
DROPOUT = .5

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 16

## Model params
h0 = 1
ACT = nn.ReLU()
LAST_ACT = nn.LogSoftmax(dim=1)
LOSS_FN = nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-A'},

        # {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
        #   'hid_dim': 16, 'leg': 'MLP'},
        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'hid_dim': 16, 'leg': 'MLP'},

        {'model': 'GAT', 'heads': 2, 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': 0,
          'hid_dim': 16, 'leg': 'GAT'},
          

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50, 'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'A-GCN-normA'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'A-GCN'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'A-GCN-v2'},

        {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .005, 'wd': 5e-4, 'drop': .5,
         'hid_dim': 32,'L': 3, 'K': 2, 'norm': True, 'leg': 'H-GCN-normH'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 50, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'bias': True, 'alpha': None, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN-alpha'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'bias': True, 'alpha': None, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'Dual-GFCN-alpha-n'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
          'bias': False, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .005, 'drop': .25,
          'bias': True, 'alpha': .8, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': .1, 'norm': True, 'leg': 'Dual-GFCN-n'},

        # {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 32,'L': 2, 'K': 3, 'norm': False, 'leg': 'H-GCN'},
         

        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN'}]

In [10]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, exp['hid_dim'], out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=exp['drop'], norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'MLP':
                arch = MLP(in_dim,  exp['hid_dim'], out_dim, dropout=exp['drop'])

            elif exp['model'] == 'GAT':
                gat_params = {'attn_drop': exp['drop']}
                arch = GAT(in_dim,  exp['hid_dim'], out_dim, exp['heads'], gat_params, act=ACT, last_act=LAST_ACT)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=exp['drop'], diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=exp['drop'], norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=exp['drop'], diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'Dual-GFGCN':
                arch = Dual_GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, alpha=exp['alpha'],
                                  last_act=LAST_ACT, dropout=exp['drop'], init_h0=exp['h0'], bias=exp['bias'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'MLP', 'GAT', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'],
                                        epochs_h=exp['e_h'], epochs_W=exp['e_W'])
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val2b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.676 (0.676) - 0.649 (0.703) - 0.649 (0.730) - 0.676 (0.676) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:44: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(np.where(np.isclose(deg_vec, 0), 0, 1/deg_vec))


0.432 (0.676) - 0.703 (0.730) - 0.378 (0.514) - 0.541 (0.595) - 0.486 (0.595) - 0.730 (0.784) - 0.865 (0.865) - 0.784 (0.865) - 0.811 (0.865) - 0.730 (0.838) - 
1: 0.568 (0.595) - 0.595 (0.649) - 0.595 (0.757) - 0.622 (0.649) - 0.865 (0.892) - 0.541 (0.676) - 0.541 (0.703) - 0.568 (0.838) - 0.541 (0.703) - 0.622 (0.703) - 0.622 (0.784) - 0.919 (0.973) - 0.919 (0.919) - 0.784 (0.892) - 
2: 0.432 (0.514) - 0.514 (0.568) - 0.486 (0.595) - 0.432 (0.486) - 0.757 (0.757) - 0.622 (0.649) - 0.351 (0.514) - 0.405 (0.459) - 0.378 (0.486) - 0.459 (0.595) - 0.838 (0.865) - 0.811 (0.892) - 0.730 (0.730) - 0.757 (0.811) - 
3: 0.595 (0.622) - 0.622 (0.622) - 0.622 (0.622) - 0.595 (0.622) - 0.865 (0.892) - 0.622 (0.649) - 0.541 (0.568) - 0.595 (0.649) - 0.541 (0.541) - 0.568 (0.757) - 0.946 (0.946) - 0.892 (0.946) - 0.838 (0.865) - 0.757 (0.784) - 
4: 0.541 (0.568) - 0.568 (0.595) - 0.568 (0.568) - 0.541 (0.568) - 0.622 (0.784) - 0.351 (0.459) - 0.432 (0.568) - 0.514 (0.622) - 0.459 (0.541) - 0.865 (0

In [11]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.578378,0.494118,0.424324,0.633114,0.8164,0.7094
Kipf-A,0.591892,0.464706,0.416216,0.290351,0.7747,0.6539
MLP,0.502703,0.354902,0.351351,0.210307,0.1610,0.1692
GAT,0.591892,0.519608,0.410811,0.587939,0.7398,0.6474
A-GCN-normA,0.727027,0.625490,0.472973,0.397588,0.7850,0.6330
A-GCN,0.581081,0.474510,0.521622,0.264254,0.6915,0.6055
A-GCN-v2,0.527027,0.456863,0.456757,0.308114,0.6821,0.5296
H-GCN-normH,0.532432,0.476471,0.375676,0.316009,0.8074,0.6839
Dual-GFCN-alpha,0.502703,0.494118,0.443243,0.414474,0.6660,0.5416
Dual-GFCN-alpha-n,0.654054,0.545098,0.567568,0.394298,0.7926,0.6563


In [12]:
table_acc_val2b

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81),CiteseerGraphDataset (0.74)
Kipf-normA,0.589189,0.539216,0.443243,0.637719,0.8159,0.7089
Kipf-A,0.594595,0.511765,0.451351,0.326096,0.7723,0.6583
MLP,0.632432,0.625490,0.427027,0.336404,0.2250,0.2579
GAT,0.556757,0.505882,0.413514,0.637281,0.7888,0.6975
A-GCN-normA,0.737838,0.682353,0.543243,0.466886,0.7946,0.6852
A-GCN,0.586486,0.533333,0.548649,0.280044,0.6984,0.6294
A-GCN-v2,0.548649,0.450980,0.518919,0.331798,0.7504,0.6342
H-GCN-normH,0.600000,0.672549,0.532432,0.327632,0.8061,0.6916
Dual-GFCN-alpha,0.505405,0.556863,0.586486,0.492763,0.7534,0.6430
Dual-GFCN-alpha-n,0.710811,0.694118,0.591892,0.468421,0.7928,0.6830
